In [1]:
using Pkg
Pkg.activate("./NRankMul")
using NRankMul
using Test

  Activating project at `~/Dev/learning/bit-tricks/nrank_multiplication/NRankMul`


$$
\boldsymbol{x}^\intercal = 
\begin{pmatrix}
x_{000} &
x_{010} &
x_{001} &
x_{011} &
x_{100} &
x_{110} &
x_{101} &
x_{111} 
\end{pmatrix}
$$



$$
\boldsymbol{M} \cdot
\begin{bmatrix}
x_{000} & x_{010} & x_{100} & x_{110} \\ 
x_{001} & x_{011} & x_{101} & x_{111}  
\end{bmatrix}
\equiv
\left( \mathbb{I}_2 \otimes \mathbb{I}_2 \otimes \boldsymbol{M}  \right)
\cdot \boldsymbol{x}
$$

$$
\boldsymbol{M} \cdot
\begin{bmatrix}
x_{000} & x_{001} & x_{100} & x_{101} \\ 
x_{010} & x_{011} & x_{110} & x_{111}  
\end{bmatrix}
\equiv
\left( \mathbb{I}_2 \otimes \boldsymbol{M} \otimes \mathbb{I}_2 \right)
\cdot \boldsymbol{x}
$$

$$
\boldsymbol{M} \cdot
\begin{bmatrix}
x_{000} & x_{001} & x_{010} & x_{011} \\ 
x_{100} & x_{101} & x_{110} & x_{111}  
\end{bmatrix}
\equiv
\left( \boldsymbol{M} \otimes \mathbb{I}_2 \otimes \mathbb{I}_2 \right)
\cdot \boldsymbol{x}
$$

$$
\begin{array}{c|c}
x_{000} & 000_2 \\
x_{001} & 001_2 \\
x_{010} & 010_2 \\
x_{011} & 011_2 \\
x_{100} & 100_2 \\
x_{101} & 101_2 \\
x_{110} & 110_2 \\
\underbrace{x_{111}}_{element} & \underbrace{111_2}_{index}
\end{array}
\xrightarrow[\text{of indices}]{\text{swap 1st and 2nd bits}}
\begin{array}{c|c}
x_{000} & 000_2 \\
x_{010} & 001_2 \\
x_{001} & 010_2 \\
x_{011} & 011_2 \\
x_{100} & 100_2 \\
x_{110} & 101_2 \\
x_{101} & 110_2 \\
x_{111} & 111_2
\end{array}
\xrightarrow[\text{of indices}]{\text{swap 1st and 3rd bits}}
\begin{array}{c|c}
x_{000} & 000_2 \\
x_{100} & 001_2 \\
x_{001} & 010_2 \\
x_{101} & 011_2 \\
x_{010} & 100_2 \\
x_{110} & 101_2 \\
x_{011} & 110_2 \\
x_{111} & 111_2
\end{array}
$$

In [2]:
fst_indices = string.(0:7; base=2, pad=3)
fst_reorder = bit_swap.(1, 0:7)
snd_indices = fst_indices[fst_reorder .+ 1]
snd_reorder = bit_swap.(2, 0:7)
trd_indices = snd_indices[snd_reorder .+ 1]

[fst_indices snd_indices trd_indices]


8×3 Matrix{String}:
 "000"  "000"  "000"
 "001"  "010"  "100"
 "010"  "001"  "001"
 "011"  "011"  "101"
 "100"  "100"  "010"
 "101"  "110"  "110"
 "110"  "101"  "011"
 "111"  "111"  "111"

In [3]:
reshape(fst_indices, 2,:) |> display
reshape(snd_indices, 2,:) |> display
reshape(trd_indices, 2,:)

2×4 Matrix{String}:
 "000"  "010"  "100"  "110"
 "001"  "011"  "101"  "111"

2×4 Matrix{String}:
 "000"  "001"  "100"  "101"
 "010"  "011"  "110"  "111"

2×4 Matrix{String}:
 "000"  "001"  "010"  "011"
 "100"  "101"  "110"  "111"

In [4]:
# Using `permutedims` instead of `transpose` since the elements are strings.
(reshape(trd_indices, 2,:) |> permutedims |> vec) == fst_indices

true

In [5]:
# Number of qubits.
N = 15

# Simulate random input.
x = rand(Float32, 2^N)

# Confusion matrix: [a₀₀ a₀₁ ; a₁₀ a₁₁].
m::Matrix{Float32} = inv([0.9 0.3; 0.1 0.7])

2×2 Matrix{Float64}:
  1.16667   -0.5
 -0.166667   1.5

In [6]:
# Using reshape.
@time res1 = multiply(m, x);

  0.002784 seconds (157 allocations: 7.754 MiB)


In [7]:
# Using Kronecker product.
@time res2::Vector{Float32} = kron(fill(m, N)...) * x;

  2.192366 seconds (698.54 k allocations: 5.368 GiB, 3.48% gc time, 7.33% compilation time)


In [8]:
@test res1 ≈ res2

Test Passed